### Data Quality Check using SQL


In [ ]:
import sqlite3
import pandas as pd
import numpy

In [7]:
conn=sqlite3.connect('case_healthverity_test01.db1')
c= conn.cursor()

c.execute('''create table claims (
                                   [claim_id] integer, 
                                   [patient_id] integer,
                                   [diagnosis_codes] text,
                                   [procedure_code] integer,
                                   [date_service] date,
                                   [date_received] date
                                   )''')

c.execute('''create table valid_icd_10_codes (
                                   [code] text)''')


c.execute('''create table valid_cpt_codes (
                                   [code] INTEGER,
                                   [short_description] text)''')
conn.commit()


In [8]:
pd.read_sql('''select * from claims''', conn)
#c.fetchall()

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received


In [9]:
claims = pd.read_csv('sample_claims.csv')
claims.to_sql('claims', conn, if_exists='append', index=False)

In [10]:
valid_icd= pd.read_csv('valid_icd_10_codes.csv')
valid_icd.to_sql('valid_icd', conn, if_exists='append', index=False)


In [15]:
pd.read_sql('''select * from valid_icd limit 5''', conn)

,code
0,A56
1,A560
2,A5600
3,A5601
4,A5602


In [12]:
valid_cpt_codes = pd.read_csv('valid_cpt_codes.csv')
valid_cpt_codes.to_sql('valid_cpt_codes', conn, if_exists='append', index=False)

In [14]:
pd.read_sql('''
                select * from claims limit 5
                ''', conn)

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
0,1,A1670,Z01.419^Z11.51,99999,1/25/2021,1/26/2021
1,2,A0086,Z01.419^Z12.4,99999,1/27/2021,1/29/2021
2,3,A0086,Z12.4^Z11.51,87491,1/7/2021,1/10/2021
3,4,A0086,R30.0,86735,1/15/2021,1/16/2021
4,5,A0311,Z34.81^I10^G40.909^E66.9,83014,1/6/2021,1/7/2021


In [17]:
pd.read_sql(''' select *
                from valid_cpt_codes limit 5 
    
                ''', conn)

,code,short_description
0,96409,CHEMO IV PUSH SNGL DRUG
1,99335,DOMICIL/R-HOME VISIT EST PAT
2,86413,SARS-COV-2 ANTB QUANTITATIVE
3,86353,LYMPHOCYTE TRANSFORMATION
4,92990,REVISION OF PULMONARY VALVE


#### Q1. Top 5 most common valid procedure codes

In [19]:
pd.read_sql(''' with temp as 
                (
                select * from claims cd 
                inner join valid_cpt_codes bc
                on cd.procedure_code = bc.code
                )
                select t.procedure_code, count(*) count
                from temp t
                group by t.procedure_code
                order by 2 desc limit 5
            
                ''', conn)

,procedure_code,count
0,88175,155
1,87591,123
2,87491,122
3,87798,116
4,85049,107


The above table consists of top 5 most common valid procedure codes based on number claims having these procedure codes. The procedure codes of the claims table is joined with valid_ procedure_codes in order populate the claims with valide procedure codes.

#### Q1.b Number of patients associated with top 5 procedure codes

In [58]:
pd.read_sql(''' with temp as 
                (
                select * from claims cd 
                inner join valid_cpt_codes bc
                on cd.procedure_code = bc.code
                )
                
                select dud.procedure_code, dud.count no_valid_procedure_code, bnb.count no_of_patients
                
                from (select t.procedure_code, count(*) count
                from temp t
                group by t.procedure_code) dud, 
                (select t.procedure_code, count(distinct t.patient_id) count
                from temp t
                group by t.procedure_code) bnb 
                where dud.procedure_code = bnb.procedure_code
                order by 2 desc limit 5
                
                
            
                ''', conn)

,procedure_code,no_valid_procedure_code,no_of_patients
0,88175,155,38
1,87591,123,31
2,87491,122,37
3,87798,116,31
4,85049,107,32


The above table represents the number of distinct patients associated with top 5 valid procedure codes.

#### Q2. Top 5 most valid diagnosis codes


In [84]:
pd.read_sql(''' WITH RECURSIVE tempA(claim_id, patient_id, diagnosis_codes, str)
                AS 
                (
                select claim_id, patient_id, '', diagnosis_codes || '^'
                from claims
                UNION all select 
                claim_id, 
                patient_id,
                substr(str, 0, instr(str, '^')),
                substr(str, instr(str, '^')+1)
                from tempA where str !=''
                
                 
                 )
                
                
                select tz.diagnosis_cd, count(*) claim_count_valid_diag_code
                from 
                (
                select claim_id, patient_id, substr(diagnosis_codes, 0, instr(diagnosis_codes, '.')) diagnosis_cd
                from tempA
                where diagnosis_codes !='') tz
                inner join valid_icd icd
                on tz.diagnosis_cd = icd.code
                group by diagnosis_cd
                order by 2 desc limit 5
                
               
            

            ''', conn)

,diagnosis_cd,claim_count_valid_diag_code
0,Z11,1474
1,Z34,861
2,Z01,529
3,E55,498
4,Z12,427


The table above represent top most 5 valid diagnosis codes based on the number of claims. The diagnosis codes of claims table 
are joined with valid icd code table.

#### Q3. Data Quality Validation

* The claim table column is validated with table description to see if columns can be nullable or not.
* Identifying number of invalid procedure codes, and diagnosis codes
* Identifying date ambiquity; date_service > date_received and date_service is in future ( not in 2021).

In [87]:
pd.read_sql('''
               
                select 
                sum(case when claim_id is null then 1 else 0 end) null_claim_id,
                sum(case when patient_id is null then 1 else 0 end) null_patient_id,
                sum(case when procedure_code is null then 1 else 0 end) null_procedure_id,
                sum(case when date_service is null then 1 else 0 end)null_service_date,
                sum(case when date_received is null then 1 else 0 end)null_date_received
                
                from claims

                ''', conn)

,null_claim_id,null_patient_id,null_procedure_id,null_service_date,null_date_received
0,2,6,3,10,0


The claim_id, patient_id, procedure_id, date_service, and date_recevied columns shouldn't not have null datapoint. The table above shows the number of null data present in each columns.

In [118]:
pd.read_sql('''
               with cte1
               as 
               (
               select substr(date_service, -4) year,  count(*) from claims
               where date_service is not null
               group by 1)
               
               select * from cte1
               
               --select count(*) from cte1 where year !='2021'

                ''', conn)
#strftime('%m',start_date) as "Month",

,year,count(*)
0,2021,4961
1,2022,4
2,2103,1
3,2124,1
4,2125,1
5,2127,1
6,2130,1
7,2133,1
8,2137,1
9,2138,1


There are total of 18 rows, which has service_date not in 2021. The invalid rows are displayed above.

In [103]:
pd.read_sql('''
                select substr(date_service, 1,2) month, count(*) from claims
                where date_service is not null
                group by 1
                ''', conn)

,month,count(*)
0,1/,4965
1,10,6
2,2/,1
3,5/,3
4,6/,1
5,7/,2
6,8/,1
7,82,3
8,84,1
9,85,1


There are total of 28 rows where month is January. These are rows are showed in the above table excluding null values. 
In additon, the table also include the rows, which don't have valid month.

In [128]:
pd.read_sql('''

          select * 
          from 
          claims tc
          
          
          
          
          where tc.procedure_code not in
          
          (
            select bc.code 
            from 
            valid_cpt_codes bc
            
              )
              
        
            ''', conn)


,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
0,1.0,A1670,Z01.419^Z11.51,99999,1/25/2021,1/26/2021
1,2.0,A0086,Z01.419^Z12.4,99999,1/27/2021,1/29/2021
2,11.0,A0311,Z00.00^E55.9,99999,1/19/2021,1/20/2021
3,12.0,A0311,Z01.419^Z11.51^N95.1,99999,1/28/2021,1/30/2021
4,14.0,A0311,N76.0,99999,1/25/2021,1/26/2021
...,...,...,...,...,...,...
1688,4977.0,A3396,N89.8^R10.2^N91.1,99999,1/6/2021,1/8/2021
1689,4979.0,A3396,Z01.419^E28.2^R53.83^E03.9,99999,1/5/2021,1/6/2021
1690,4981.0,A3396,Z00.00^Z11.3^Z11.59^Z13.228^Z13.29^Z13.79^Z13....,99999,1/4/2021,1/8/2021
1691,4989.0,A3396,Z11.3,Z11.8,10/13/2173,1/11/2021


There are 1693 rows, which do not have valid procedure code. The above table represents rows having invalid procedure_code

In [142]:
pd.read_sql(''' WITH RECURSIVE tempA(claim_id, patient_id, diagnosis_codes, str)
                AS 
                (
                select claim_id, patient_id, '', diagnosis_codes || '^'
                from claims
                UNION all select 
                claim_id, 
                patient_id,
                substr(str, 0, instr(str, '^')),
                substr(str, instr(str, '^')+1)
                from tempA where str !=''
                
                 
                 )
                
                ---select count(*)
                --from 
                
                --(
                
                select tz.*--icd.code code2
                from 
                (
                select claim_id, patient_id, substr(diagnosis_codes, 0, instr(diagnosis_codes, '.')) diagnosis_cd
                from tempA
                where diagnosis_codes!='')tz
                
                where tz.diagnosis_cd not in (select code from valid_icd)
                
                --left join valid_icd icd
                --on tz.diagnosis_cd = icd.code) z
                --where z.code2 is null
                --group by diagnosis_cd
                --order by 2 desc limit 5
                
               
            

            ''', conn)

,claim_id,patient_id,diagnosis_cd
0,101,A0311,Z87
1,113,A0482,
2,130,A0482,
3,225,A1245,Z87
4,273,A1249,
...,...,...,...
435,4860,A3379,
436,4889,A3379,
437,4892,A3379,Z86
438,4932,A3388,


There are 440 invaide diagnosis code in the claims table. The table above shows the row with invalid diagnosis code.

In [124]:
pd.read_sql('''

          select tv.* 
          --- select count(*)
          from claims tv
          where tv.diagnosis_codes not in
          
          (
            select bc.code 
            from 
            valid_icd bc
            
              )
              
        
            ''', conn)

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
0,1.0,A1670,Z01.419^Z11.51,99999,1/25/2021,1/26/2021
1,2.0,A0086,Z01.419^Z12.4,99999,1/27/2021,1/29/2021
2,3.0,A0086,Z12.4^Z11.51,87491,1/7/2021,1/10/2021
3,4.0,A0086,R30.0,86735,1/15/2021,1/16/2021
4,5.0,A0311,Z34.81^I10^G40.909^E66.9,83014,1/6/2021,1/7/2021
...,...,...,...,...,...,...
3665,4997.0,A3396,N90.89,87491,1/29/2021,2/6/2021
3666,4999.0,A3396,Z83.3^R63.5^Z11.3^E55.9,80307,1/27/2021,1/28/2021
3667,5000.0,A3396,Z01.419,84132,1/3/2021,1/5/2021
3668,NaN,A1623,N92.6^Z11.3^Z11.8^Z32.01,99999,1/4/2021,1/9/2021


In [146]:
 #Let check if there are rows where date_service is greater than date_received.

#pd.read_sql('''
                #select * 
                #from claims
                #where strftime(date_service, '%Y/%m/%d'), strftime(date_service, '%Y/%m/%d')
                #date_service is not null
                #''', conn)



From the Python findings, there are 81 rows in claims table where date_service is greater than date_received. 